In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
from tensorflow import keras
import pandas as pd

print(tf.__version__)

D:\Applications\anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


1.11.0


# 数据预处理

# 数据加载

In [2]:
train_data=pd.read_csv("D:\\jupterWork\\encoder\\ag_news_csv\\train.csv",header=None,usecols=[1,2])
train_labels=pd.read_csv("D:\\jupterWork\\encoder\\ag_news_csv\\train.csv",header=None,usecols=[0])
test_data=pd.read_csv("D:\\jupterWork\\encoder\\ag_news_csv\\test.csv",header=None,usecols=[1,2])
test_labels=pd.read_csv("D:\\jupterWork\\encoder\\ag_news_csv\\test.csv",header=None,usecols=[0])

# titlte+description

In [3]:
#array
train_data=train_data.values
y_train=train_labels.values
test_data=test_data.values
y_test=test_labels.values

train_messages = [None]*120000
test_messages = [None]*7600

for index, data in enumerate(train_data):
    train_messages[index] = data[0]+'.'+data[1]

for index, data in enumerate(test_data):
    test_messages[index] = data[0]+'.'+data[1]

# 向量模型

In [4]:
paths="D:\\jupterWork\\tfhub_modules\\1fb57c3ffe1a38479233ee9853ddd7a8ac8a8c47"
# Import the Universal Sentence Encoder's TF Hub module
embed = hub.Module(paths)

INFO:tensorflow:Using C:\Users\Dream\AppData\Local\Temp\tfhub_modules to cache modules.


# encoder句子转化向量

In [5]:
# Reduce logging output.
tf.logging.set_verbosity(tf.logging.ERROR)
with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    x_train = session.run(embed(train_messages))
    x_test = session.run(embed(test_messages))

# 构建模型

In [131]:
# 
from tensorflow.keras.layers import LSTM, Dense
model = keras.Sequential()
model.add(Dense(128, input_dim=512))
model.add(Dense(32, activation='relu'))
model.add(Dense(5, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_32 (Dense)             (None, 128)               65664     
_________________________________________________________________
dense_33 (Dense)             (None, 32)                4128      
_________________________________________________________________
dense_34 (Dense)             (None, 5)                 165       
Total params: 69,957
Trainable params: 69,957
Non-trainable params: 0
_________________________________________________________________


In [132]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [133]:
y_train_labels = keras.utils.to_categorical(y_train, num_classes=5)
y_train_labels

array([[0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       ...,
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.]], dtype=float32)

In [134]:
model.fit(x_train, y_train_labels, epochs=5,callbacks=[TensorBoard(log_dir='./tmp/log')])

Epoch 1/5
120000/120000 [==============================] - 197s 2ms/step - loss: 0.3143 - acc: 0.8920
Epoch 2/5
120000/120000 [==============================] - 205s 2ms/step - loss: 0.2903 - acc: 0.8986
Epoch 3/5
120000/120000 [==============================] - 210s 2ms/step - loss: 0.2832 - acc: 0.9010
Epoch 4/5
120000/120000 [==============================] - 196s 2ms/step - loss: 0.2783 - acc: 0.9031
Epoch 5/5
120000/120000 [==============================] - 205s 2ms/step - loss: 0.2741 - acc: 0.9046


In [139]:
y_test_labels = keras.utils.to_categorical(y_test, num_classes=5)
results = model.evaluate(x_test, y_test_labels)
print(results)

7600/7600 [==============================] - 2s 310us/step
[0.2994171789445375, 0.8971052631578947]


In [148]:
predictions = model.predict(x_test)
for index,pre in enumerate(predictions):
    print('real:',y_test[index][0])
    print('   pre:',np.argmax(pre))

real: 3
   pre: 3
real: 4
   pre: 4
real: 4
   pre: 4
real: 4
   pre: 3
real: 4
   pre: 4
real: 4
   pre: 4
real: 4
   pre: 4
real: 4
   pre: 4
real: 4
   pre: 4
real: 4
   pre: 3
real: 4
   pre: 4
real: 4
   pre: 4
real: 4
   pre: 4
real: 4
   pre: 4
real: 4
   pre: 4
real: 4
   pre: 1
real: 4
   pre: 4
real: 4
   pre: 4
real: 4
   pre: 4
real: 4
   pre: 3
real: 4
   pre: 4
real: 4
   pre: 4
real: 4
   pre: 4
real: 4
   pre: 3
real: 4
   pre: 3
real: 4
   pre: 4
real: 2
   pre: 2
real: 2
   pre: 2
real: 2
   pre: 2
real: 2
   pre: 2
real: 2
   pre: 2
real: 2
   pre: 2
real: 1
   pre: 1
real: 1
   pre: 1
real: 1
   pre: 1
real: 1
   pre: 1
real: 1
   pre: 3
real: 1
   pre: 1
real: 1
   pre: 1
real: 1
   pre: 1
real: 2
   pre: 2
real: 3
   pre: 3
real: 1
   pre: 1
real: 4
   pre: 4
real: 2
   pre: 2
real: 2
   pre: 2
real: 1
   pre: 1
real: 4
   pre: 4
real: 1
   pre: 1
real: 2
   pre: 2
real: 1
   pre: 1
real: 2
   pre: 2
real: 1
   pre: 1
real: 4
   pre: 3
real: 3
   pre: 3
real: 4
  

real: 1
   pre: 1
real: 1
   pre: 3
real: 1
   pre: 1
real: 2
   pre: 2
real: 4
   pre: 4
real: 4
   pre: 4
real: 4
   pre: 4
real: 3
   pre: 3
real: 3
   pre: 3
real: 4
   pre: 4
real: 4
   pre: 4
real: 2
   pre: 2
real: 4
   pre: 4
real: 2
   pre: 2
real: 1
   pre: 1
real: 1
   pre: 1
real: 4
   pre: 4
real: 2
   pre: 2
real: 4
   pre: 4
real: 3
   pre: 3
real: 2
   pre: 2
real: 4
   pre: 1
real: 3
   pre: 3
real: 1
   pre: 1
real: 1
   pre: 1
real: 1
   pre: 1
real: 2
   pre: 2
real: 4
   pre: 3
real: 4
   pre: 4
real: 4
   pre: 4
real: 4
   pre: 4
real: 4
   pre: 4
real: 4
   pre: 4
real: 4
   pre: 4
real: 1
   pre: 1
real: 3
   pre: 3
real: 4
   pre: 4
real: 2
   pre: 2
real: 1
   pre: 3
real: 1
   pre: 1
real: 4
   pre: 4
real: 4
   pre: 3
real: 2
   pre: 2
real: 4
   pre: 4
real: 3
   pre: 2
real: 4
   pre: 4
real: 4
   pre: 3
real: 1
   pre: 4
real: 3
   pre: 3
real: 2
   pre: 2
real: 4
   pre: 4
real: 4
   pre: 4
real: 1
   pre: 1
real: 2
   pre: 2
real: 4
   pre: 4
real: 3
  

   pre: 4
real: 4
   pre: 4
real: 1
   pre: 1
real: 3
   pre: 3
real: 3
   pre: 3
real: 4
   pre: 3
real: 4
   pre: 4
real: 1
   pre: 1
real: 2
   pre: 2
real: 4
   pre: 4
real: 3
   pre: 4
real: 4
   pre: 4
real: 4
   pre: 3
real: 1
   pre: 1
real: 2
   pre: 2
real: 1
   pre: 1
real: 4
   pre: 3
real: 4
   pre: 1
real: 3
   pre: 3
real: 4
   pre: 4
real: 4
   pre: 4
real: 1
   pre: 1
real: 2
   pre: 2
real: 1
   pre: 1
real: 3
   pre: 3
real: 3
   pre: 4
real: 1
   pre: 1
real: 1
   pre: 1
real: 2
   pre: 2
real: 2
   pre: 2
real: 2
   pre: 2
real: 4
   pre: 4
real: 1
   pre: 1
real: 4
   pre: 4
real: 2
   pre: 2
real: 1
   pre: 1
real: 3
   pre: 3
real: 3
   pre: 3
real: 1
   pre: 1
real: 1
   pre: 1
real: 4
   pre: 3
real: 4
   pre: 1
real: 3
   pre: 3
real: 2
   pre: 2
real: 4
   pre: 4
real: 4
   pre: 4
real: 4
   pre: 4
real: 4
   pre: 3
real: 4
   pre: 3
real: 4
   pre: 4
real: 4
   pre: 4
real: 3
   pre: 3
real: 4
   pre: 4
real: 4
   pre: 3
real: 1
   pre: 2
real: 2
   pre: 2


real: 4
   pre: 4
real: 3
   pre: 3
real: 1
   pre: 1
real: 1
   pre: 1
real: 3
   pre: 3
real: 3
   pre: 3
real: 1
   pre: 1
real: 1
   pre: 2
real: 4
   pre: 4
real: 1
   pre: 1
real: 2
   pre: 2
real: 1
   pre: 1
real: 1
   pre: 1
real: 3
   pre: 3
real: 2
   pre: 2
real: 4
   pre: 3
real: 4
   pre: 4
real: 1
   pre: 1
real: 1
   pre: 1
real: 1
   pre: 1
real: 1
   pre: 1
real: 2
   pre: 2
real: 3
   pre: 3
real: 1
   pre: 1
real: 4
   pre: 4
real: 2
   pre: 2
real: 3
   pre: 3
real: 2
   pre: 2
real: 2
   pre: 2
real: 2
   pre: 2
real: 3
   pre: 3
real: 2
   pre: 2
real: 2
   pre: 2
real: 1
   pre: 1
real: 2
   pre: 2
real: 3
   pre: 3
real: 1
   pre: 1
real: 2
   pre: 2
real: 2
   pre: 2
real: 2
   pre: 2
real: 2
   pre: 2
real: 1
   pre: 1
real: 1
   pre: 1
real: 2
   pre: 2
real: 2
   pre: 2
real: 2
   pre: 2
real: 1
   pre: 1
real: 4
   pre: 4
real: 2
   pre: 2
real: 3
   pre: 3
real: 1
   pre: 1
real: 2
   pre: 2
real: 1
   pre: 1
real: 4
   pre: 4
real: 1
   pre: 1
real: 1
  

   pre: 4
real: 2
   pre: 2
real: 1
   pre: 1
real: 1
   pre: 2
real: 1
   pre: 1
real: 1
   pre: 1
real: 2
   pre: 2
real: 2
   pre: 2
real: 3
   pre: 3
real: 4
   pre: 4
real: 3
   pre: 3
real: 2
   pre: 2
real: 1
   pre: 1
real: 3
   pre: 3
real: 2
   pre: 2
real: 2
   pre: 2
real: 4
   pre: 4
real: 1
   pre: 1
real: 1
   pre: 1
real: 2
   pre: 2
real: 2
   pre: 2
real: 3
   pre: 3
real: 1
   pre: 1
real: 2
   pre: 2
real: 3
   pre: 3
real: 2
   pre: 2
real: 2
   pre: 2
real: 2
   pre: 2
real: 1
   pre: 3
real: 1
   pre: 1
real: 1
   pre: 1
real: 1
   pre: 1
real: 1
   pre: 1
real: 2
   pre: 2
real: 1
   pre: 1
real: 1
   pre: 1
real: 4
   pre: 4
real: 3
   pre: 4
real: 1
   pre: 1
real: 2
   pre: 2
real: 1
   pre: 1
real: 2
   pre: 2
real: 3
   pre: 3
real: 3
   pre: 3
real: 2
   pre: 2
real: 4
   pre: 4
real: 1
   pre: 1
real: 2
   pre: 2
real: 1
   pre: 1
real: 1
   pre: 1
real: 2
   pre: 2
real: 1
   pre: 1
real: 2
   pre: 2
real: 2
   pre: 2
real: 2
   pre: 2
real: 3
   pre: 3


...

real: 2
   pre: 2
real: 3
   pre: 3
real: 3
   pre: 3
real: 4
   pre: 4
real: 4
   pre: 4
real: 1
   pre: 1
real: 3
   pre: 3
real: 2
   pre: 2
real: 3
   pre: 3
real: 4
   pre: 4
real: 3
   pre: 4
real: 4
   pre: 4
real: 1
   pre: 1
real: 3
   pre: 3
real: 2
   pre: 2
real: 4
   pre: 4
real: 2
   pre: 2
real: 1
   pre: 1
real: 3
   pre: 3
real: 3
   pre: 3
real: 3
   pre: 3
real: 2
   pre: 2
real: 1
   pre: 1
real: 4
   pre: 4
real: 4
   pre: 4
real: 2
   pre: 2
real: 3
   pre: 3
real: 1
   pre: 1
real: 1
   pre: 1
real: 4
   pre: 4
real: 2
   pre: 2
real: 1
   pre: 1
real: 3
   pre: 3
real: 4
   pre: 4
real: 4
   pre: 4
real: 2
   pre: 2
real: 4
   pre: 4
real: 3
   pre: 3
real: 2
   pre: 2
real: 1
   pre: 1
real: 3
   pre: 3
real: 2
   pre: 2
real: 1
   pre: 1
real: 4
   pre: 4
real: 1
   pre: 1
real: 4
   pre: 4
real: 1
   pre: 1
real: 2
   pre: 2
real: 2
   pre: 2
real: 2
   pre: 2
real: 2
   pre: 2
real: 3
   pre: 3
real: 3
   pre: 3
real: 1
   pre: 1
real: 2
   pre: 2
real: 2
  

   pre: 3
real: 3
   pre: 4
real: 3
   pre: 3
real: 3
   pre: 3
real: 3
   pre: 3
real: 4
   pre: 4
real: 3
   pre: 3
real: 3
   pre: 4
real: 2
   pre: 2
real: 4
   pre: 4
real: 4
   pre: 4
real: 1
   pre: 1
real: 2
   pre: 2
real: 4
   pre: 4
real: 4
   pre: 4
real: 4
   pre: 4
real: 3
   pre: 3
real: 4
   pre: 4
real: 2
   pre: 2
real: 2
   pre: 2
real: 1
   pre: 1
real: 3
   pre: 3
real: 3
   pre: 4
real: 1
   pre: 1
real: 1
   pre: 1
real: 1
   pre: 1
real: 1
   pre: 1
real: 1
   pre: 1
real: 3
   pre: 3
real: 3
   pre: 3
real: 3
   pre: 3
real: 2
   pre: 2
real: 3
   pre: 3
real: 3
   pre: 3
real: 4
   pre: 4
real: 2
   pre: 2
real: 2
   pre: 2
real: 1
   pre: 3
real: 4
   pre: 4
real: 2
   pre: 2
real: 1
   pre: 1
real: 4
   pre: 4
real: 4
   pre: 1
real: 3
   pre: 3
real: 1
   pre: 1
real: 3
   pre: 3
real: 3
   pre: 3
real: 4
   pre: 3
real: 3
   pre: 3
real: 3
   pre: 3
real: 4
   pre: 4
real: 3
   pre: 3
real: 2
   pre: 2
real: 3
   pre: 3
real: 1
   pre: 1
real: 2
   pre: 2


real: 3
   pre: 3
real: 2
   pre: 2
real: 1
   pre: 1
real: 3
   pre: 3
real: 2
   pre: 2
real: 1
   pre: 1
real: 2
   pre: 2
real: 2
   pre: 2
real: 2
   pre: 2
real: 2
   pre: 2
real: 4
   pre: 4
real: 3
   pre: 3
real: 2
   pre: 2
real: 2
   pre: 2
real: 3
   pre: 1
real: 2
   pre: 2
real: 2
   pre: 2
real: 2
   pre: 2
real: 4
   pre: 3
real: 3
   pre: 3
real: 2
   pre: 2
real: 3
   pre: 4
real: 1
   pre: 1
real: 1
   pre: 1
real: 1
   pre: 1
real: 2
   pre: 2
real: 2
   pre: 2
real: 2
   pre: 2
real: 1
   pre: 1
real: 2
   pre: 2
real: 1
   pre: 1
real: 4
   pre: 4
real: 2
   pre: 2
real: 3
   pre: 3
real: 3
   pre: 3
real: 1
   pre: 1
real: 4
   pre: 4
real: 4
   pre: 4
real: 1
   pre: 1
real: 4
   pre: 4
real: 3
   pre: 3
real: 4
   pre: 4
real: 4
   pre: 4
real: 1
   pre: 1
real: 3
   pre: 4
real: 3
   pre: 3
real: 4
   pre: 4
real: 4
   pre: 4
real: 2
   pre: 2
real: 2
   pre: 2
real: 4
   pre: 4
real: 3
   pre: 4
real: 2
   pre: 2
real: 3
   pre: 4
real: 4
   pre: 4
real: 1
  

   pre: 3
real: 1
   pre: 1
real: 1
   pre: 1
real: 4
   pre: 4
real: 1
   pre: 1
real: 1
   pre: 1
real: 3
   pre: 3
real: 4
   pre: 4
real: 3
   pre: 3
real: 1
   pre: 1
real: 3
   pre: 1
real: 4
   pre: 4
real: 1
   pre: 4
real: 3
   pre: 1
real: 1
   pre: 1
real: 4
   pre: 4
real: 4
   pre: 3
real: 1
   pre: 1
real: 4
   pre: 4
real: 3
   pre: 3
real: 4
   pre: 4
real: 1
   pre: 1
real: 3
   pre: 3
real: 3
   pre: 3
real: 4
   pre: 4
real: 3
   pre: 3
real: 3
   pre: 3
real: 2
   pre: 2
real: 4
   pre: 4
real: 4
   pre: 4
real: 3
   pre: 3
real: 3
   pre: 4
real: 4
   pre: 4
real: 1
   pre: 1
real: 4
   pre: 4
real: 4
   pre: 3
real: 3
   pre: 3
real: 3
   pre: 3
real: 1
   pre: 1
real: 4
   pre: 4
real: 4
   pre: 4
real: 3
   pre: 3
real: 4
   pre: 4
real: 1
   pre: 1
real: 2
   pre: 2
real: 1
   pre: 3
real: 2
   pre: 2
real: 4
   pre: 4
real: 4
   pre: 3
real: 2
   pre: 2
real: 3
   pre: 3
real: 1
   pre: 1
real: 2
   pre: 2
real: 1
   pre: 1
real: 3
   pre: 3
real: 3
   pre: 3


real: 4
   pre: 4
real: 4
   pre: 4
real: 4
   pre: 4
real: 3
   pre: 3
real: 3
   pre: 3
real: 2
   pre: 2
real: 1
   pre: 1
real: 4
   pre: 3
real: 2
   pre: 2
real: 1
   pre: 1
real: 3
   pre: 3
real: 2
   pre: 2
real: 2
   pre: 2
real: 1
   pre: 1
real: 1
   pre: 1
real: 3
   pre: 3
real: 1
   pre: 1
real: 3
   pre: 3
real: 1
   pre: 1
real: 1
   pre: 4
real: 1
   pre: 1
real: 4
   pre: 4
real: 4
   pre: 3
real: 3
   pre: 3
real: 1
   pre: 1
real: 1
   pre: 1
real: 4
   pre: 4
real: 3
   pre: 3
real: 3
   pre: 3
real: 3
   pre: 3
real: 2
   pre: 2
real: 2
   pre: 2
real: 1
   pre: 1
real: 3
   pre: 3
real: 3
   pre: 3
real: 3
   pre: 3
real: 3
   pre: 3
real: 3
   pre: 3
real: 3
   pre: 3
real: 4
   pre: 4
real: 1
   pre: 1
real: 3
   pre: 3
real: 2
   pre: 2
real: 2
   pre: 2
real: 3
   pre: 3
real: 4
   pre: 4
real: 4
   pre: 3
real: 4
   pre: 3
real: 1
   pre: 1
real: 4
   pre: 4
real: 1
   pre: 1
real: 2
   pre: 2
real: 2
   pre: 2
real: 4
   pre: 4
real: 4
   pre: 4
real: 2
  